In [ ]:
import pandas as pd
import numpy as np
import psycopg2 as ps

# Create function

In [ ]:
df = pd.read_csv(r'FILE', index_col=0) 

In [ ]:
def insert_into_table(curr, shop_id, id, status_name, status, tags, inserted_at, id_column,  updated_at, revenue, warehouse_info, status_history, 
                       partner, assigning_seller):
    insert_into_records = ("""INSERT INTO project_1_data (shop_id, id, status_name, status, tags, inserted_at, id_column,  updated_at, revenue, warehouse_info, status_history, 
     partner, assigning_seller)
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""")
    row_to_insert = (shop_id, id, status_name, status, tags, inserted_at, id_column,  updated_at, revenue , warehouse_info, status_history, 
                      partner, assigning_seller)
    curr.execute(insert_into_records, row_to_insert)


def update_row(curr, shop_id, id, status_name, status, tags, inserted_at, id_column,  updated_at, revenue, warehouse_info, status_history, 
                partner, assigning_seller):
    query = ("""UPDATE project_1_data
            SET shop_id                 = %s,
                id                      = %s,
                status_name             = %s,
                status                  = %s,
                tags                    = %s,
                inserted_at             = %s,
                updated_at              = %s, 
                revenue                 = %s,
                warehouse_info          = %s,
                status_history          = %s,
                partner                 = %s,
                assigning_seller        = %s
            WHERE id_column             = %s;""")
    vars_to_update = (shop_id, id, status_name, status, tags, inserted_at,  updated_at, revenue, warehouse_info, status_history, 
                       partner, assigning_seller, id_column)
    curr.execute(query, vars_to_update)


def check_if_record_exists(curr, id_column): 
    query = ("""SELECT id_column FROM project_1_data WHERE id_column = %s""")

    curr.execute(query, (id_column,))
    return curr.fetchone() is not None


def append_from_df_to_db(curr,df2):
    for i, row in df2.iterrows():
        insert_into_table(curr, row['shop_id'], row['id'], row['status_name'], row['status'], row['tags'], row['inserted_at'], row['id_column'], row['updated_at'], row['revenue'], row['warehouse_info'], row['status_history'], 
                          row[''], row['partner'], row['assigning_seller'])


def update_db(curr,df2):
    tmp_df = pd.DataFrame(columns=['shop_id', 'id', 'status_name', 'status', 'tags', 'inserted_at', 'id_column',  'updated_at', 'revenue',  'warehouse_info', 'status_history', 
                                    'partner', 'assigning_seller'])
    for i, row in df2.iterrows():
        if check_if_record_exists(curr, row['id_column']): 
            update_row(curr, row['shop_id'], row['id'], row['status_name'], row['status'], row['tags'], row['inserted_at'], row['id_column'],   row['updated_at'], row['revenue'],  row['warehouse_info'], row['status_history'], 
                        row['partner'], row['assigning_seller'])
        else: 
            tmp_df = pd.concat([tmp_df, pd.DataFrame([row])])

    return tmp_df

#____________________________________________________________________________________________________________________________________________________________________________________________________________

# Kết nối với database
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host = host_name, database = dbname, 
               user = username, password = password, port = port)
    except ps.OperationalError as e:
        raise e
    else:
        print('Hurrayyyyyy! I am on the CLOUD!')
        return conn

# Connect to database

In [ ]:
host_name          = ''
dbname             = ''
port               = ''
username           = ''
password           = ''
conn               = None

conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

# Divide data into small chunk, upload them

In [ ]:
number_of_chunk = len(df) / 100
dfs = np.array_split(df, int(number_of_chunk) +1)

# Edit (To assign a new col based on sequential number of df in dfs):

print('- Number of pieces in df is: ' + str(len(dfs)))

chunk_list_df = pd.read_csv(r'FILE', index_col=0) 
print('- Number of Chunk is: ' + str(len(chunk_list_df)))

In [ ]:
if len(chunk_list_df) == 0:
    print('Congratulation, sir!')
else:
    for i in chunk_list_df['Chunk']:
        print('Uploading _ _ _ _ _ _ _ _ ' + str(chunk_list_df['Chunk'][i]))
        new_vid_df = update_db(curr, dfs[i])
        conn.commit()
    
        append_from_df_to_db(curr, new_vid_df)
        conn.commit()
    
        print('Drop '+ str(chunk_list_df['Chunk'][i]))
        chunk_list_df = chunk_list_df.drop(chunk_list_df['Chunk'][i])
        chunk_list_df.to_csv(r'FILE')
        print('')